In [3]:
import json
import pandas as pd
from tools.project import OUTPUT_PATH

In [4]:
def load_metrics(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    df = pd.DataFrame.from_dict(data, orient='index')
    return df

,fad_ti,fad_other,clap_ti,clap_other,kncc_ti,knco_ti,kncc_other,knco_other
two-steps,10.336038,8.334016,17.603073,16.204828,0.052050,0.052050,0.046284,0.000000
chillout,5.954008,6.997991,16.040289,16.345755,0.144632,0.144632,0.152855,0.000000
metal-solos,12.999258,15.121348,21.543287,20.393539,0.055435,0.055435,0.055329,0.000000
8bit-slow,9.005074,5.708650,17.677357,15.793801,0.390754,0.390754,0.384925,0.000000
choir,10.271920,11.204199,13.969916,10.479802,0.358392,0.358392,0.322613,0.166667
8bit,10.857361,8.091739,18.285793,18.321852,0.798995,0.798995,0.716918,0.000000
piano,14.197317,12.910067,14.305449,15.620468,0.112451,0.112451,0.140145,0.000000
80s-synth,8.269018,7.310723,11.422831,12.405912,0.076159,0.076159,0.096482,0.000000
saxophone-chillout,8.394822,5.857701,19.438158,18.905127,0.090285,0.090285,0.087186,0.000000
b-minor-rock,15.450402,7.728103,17.825686,18.733162,0.067019,0.067019,0.072256,0.000000


In [5]:
def rename_columns(df):
    """
    Rename columns to use LaTeX-friendly names.
    For example, fad_ti -> FAD$_{ti}$, kncc_other -> kNCC$_{other}$, etc.
    """
    column_map = {
        "fad_ti":      r"FAD$_{ti}$",
        "fad_other":   r"FAD$_{other}$",
        "clap_ti":     r"CLAP$_{ti}$",
        "clap_other":  r"CLAP$_{other}$",
        "kncc_ti":     r"kNCC$_{ti}$",
        "knco_ti":     r"kNCO$_{ti}$",
        "kncc_other":  r"kNCC$_{other}$",
        "knco_other":  r"kNCO$_{other}$"
    }
    return df.rename(columns=column_map)

In [9]:
metrics_df = load_metrics(OUTPUT_PATH('comparison', 'ti-musicgen-clean.json'))[['fad_ti', 'clap_ti', 'kncc_ti', 'knco_ti']]
metrics_df = rename_columns(metrics_df)
metrics_df = metrics_df.round(3)
latex_str = metrics_df.to_latex(
        index=True,            # Keep the row labels (the category names)
        escape=False,          # Important to allow LaTeX syntax in the column names
        float_format="%.3f",
        caption="Metrics Table with Mean and Variance",  # Example caption
        label="tab:metrics"    # Example label
    )
    
with open(OUTPUT_PATH('visualization', "metrics_table.tex"), "w") as f:
    f.write(latex_str)